# Assignment 3

### Submitted by Udayaditya Pratap Singh, Saransh Gupta, Neel Shah, Lakshya Sharma, Rishabh Surana

##### 17BT10026 , 17QM30005, 17QE30006, 17NA30026, 17MT30016

Meta Info
1. Two files are attached which contain daily rainfall data over India for 2010 and 2011.

2. Both contain 357x122 matrix (XR1 and XR) and a binary vector (ZR1 and ZR).

3. Rainfall amounts at 357 locations over India, on each day during the monsoon seasons of 2010 and 2011 (122 days from 1 June to 30 September).

4. ZR1 and ZR are binary vectors which classify every day as 'rainy" (1) or non-rainy (0) based on the rainfall across the landmass.

In [84]:
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn import linear_model as pLM
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

import scipy.io

# 1. Read the .mat files in Python and access the variables

In [ ]:
def make_readable(path):
    """
    path -- i/p string of the path of the .mat file

    Return -- Readable Table w/ Locations as rows, dates as column
              ## --> First Row = Rainy day or not in India (avgd. over locations)
    """

    # Reading the .mat data
    dat_rainfall = scipy.io.loadmat(str(path))
    XR = list(dat_rainfall.keys())[-2]
    ZR = list(dat_rainfall.keys())[-1]

    # XR1 - amount of rainfall at different locations (rows) on different days (columns)
    index_XR1 = ['Pos ' + str(day) for day in range(dat_rainfall[str(XR)].shape[0])]
    cols_XR1 = ["Day" + str(pos_) for pos_ in range(dat_rainfall[str(XR)].shape[1])]

    dat_XR1 = pd.DataFrame(data=dat_rainfall[str(XR)],
                            index=index_XR1, columns=cols_XR1)

    # ZR1 - classification of location as Rainy/Non-Rainy
    cols_ZR1 = ['Day' + str(day) for day in range(dat_rainfall[str(ZR)].shape[1])]
    index_ZR1 = ["Rainy / Not"]

    dat_ZR1 = pd.DataFrame(data=dat_rainfall[str(ZR)],
                                index=index_ZR1, columns=cols_ZR1)

    table_ = dat_ZR1.copy()
    table_ = table_.append(dat_XR1)
    table_.iloc[0].apply(int)

    return table_.T

In [68]:
# table_2010 = make_readable("/content/2010rainfall.mat")
# table_2011 = make_readable("/content/2011rainfall.mat")

table_2010 = make_readable("2010rainfall.mat")
table_2011 = make_readable("2011rainfall.mat")

"""
## Uncomment to show
xx = table_2010.drop(['Pos 1'], axis = 1)
np.array(xx)
print(xx)
"""
print(table_2010)
print(table_2011)

        Rainy / Not  Pos 0  Pos 1  ...  Pos 354  Pos 355  Pos 356
Day0            0.0    0.0    0.0  ...      0.0      0.0      0.0
Day1            0.0    0.0    0.0  ...      0.0      0.0      0.0
Day2            0.0    0.0    0.0  ...      0.0      0.0      0.0
Day3            0.0    0.0    0.0  ...      0.0      0.0      0.0
Day4            0.0    0.0    0.0  ...      0.0      0.0      0.0
...             ...    ...    ...  ...      ...      ...      ...
Day117          0.0    0.0    0.0  ...      0.0      0.0      0.0
Day118          0.0    0.0    0.0  ...      0.0      0.0      0.0
Day119          0.0    0.0    0.0  ...      0.0      0.0      0.0
Day120          0.0    0.0    0.0  ...      0.0      0.0      0.0
Day121          0.0    0.0    0.0  ...      0.0      0.0      0.0

[122 rows x 358 columns]
        Rainy / Not  Pos 0  Pos 1  ...    Pos 354  Pos 355    Pos 356
Day0            0.0    0.0    0.0  ...  34.099998      0.0  17.049999
Day1            0.0    0.0    0.0  ...  12

# Linear regression

Use a linear regression model to predict the rainfall **XR(s,t)** at any location 's' on day 't', 

__Predictor:__

---> using the rainfall at all other locations on the same day, and also rainfall at the same location on the previous 2 days 

[XR(1,t)....XR(s-1,t), XR(s+1,t),....XR(357,t), XR(s,t-1), XR(s,t-2)].

***Use 2010 data for training***

Build such a model for s=42 (Mumbai), s=158 (Delhi), s= 299 (Kharagpur) [3 marks]

In [45]:
def linReg_output(city, train=table_2010, test=table_2011, day=None,
                  print_=False, debug=False):
    """Input
    city -- the numerical index of the city for which linear model
    has to be applied
    train -- training data, 2010 by default
    test -- testing data, 2011 by default
    day -- predicted rainfall for specific day. Prints for all day by default
           values fromm 0 to 121, whole numbers
    print_ -- False by deafault, to directly output the predicted values

    Output
    prints the predicted
    Returns an array of the predicted value, or the predicted value for a
    particular day
    """

    position = "Pos " + str(city)
    model = LinearRegression()

    y_train = train[position]
    y_test = test[position]

    model.fit(table_2010.drop(["Rainy / Not", position], axis = 1),
              y_train)
    y_pred = model.predict(table_2011.drop(["Rainy / Not", position], axis = 1))
    
    # convert negative values to zeroes -- since rainfall cannot be -ve
    y_pred[y_pred < 0] = 0

    if debug:
        print("Y train:")
        print(y_train)
        print("Y Test:")
        print(y_test)

    # predicted rainfall on that position using rainfall at other areas
    if day == None:
        if print_:
            print(y_pred)
        return y_pred
    elif day < 121 and day >= 0:
        if print_:
            print(y_pred[day])
        return y_pred[day]


## Mumbai

In [46]:
predictionM = linReg_output(42, print_=True)  # Mumbai

#predictionMD = linReg_output(42, day=0)  # predicted rainfall on specific day

[  0.          26.04124861  27.04342057  29.72509527  37.90413453
  38.19533216  11.19751914   0.           3.67325741  33.46861844
  27.48747153  85.05117479  21.80118699   0.          38.10481485
  26.71181782  39.23433859   4.20465102   4.57621885  32.67185245
  12.24020787   0.          92.49502431  34.0030677   19.52161146
   5.59918458  23.19852267  31.57620104  39.20122693   0.
   0.           0.          17.74660184   2.51062485   6.66940471
   0.          31.96901642  85.57671459  59.38005476  50.96271968
  39.80882305  37.07181237  55.24995741  97.68432386  95.98351282
  54.04706493 100.94082109  71.62331288  54.85509708  59.70307215
   0.           5.49150421  30.33115338  24.20773468  29.89277014
  36.86064519  25.43139535  16.45919801  78.21916037  97.30600229
 145.80589301  80.54401439  43.86289217  22.7227939   42.34832559
  45.42768336  34.74324285   8.72220483  25.3031582   33.74768745
  13.11092218  21.76388768  23.65945479  40.44990865  15.10239502
   0.          10.

## Delhi

In [49]:
predictionD = linReg_output(158, print_=True)  # Delhi

[ 0.          0.          3.19605794  4.66160967  1.83962681  0.19076494
  0.          4.15009866  0.          2.8750577   5.08696452  5.32043195
  0.          6.89885101  5.8164108  13.36282089  4.26787851 21.46575303
 34.65605763 32.92636666 14.93005213  0.69108023 14.84077762  7.16741624
  1.60346721 11.85368041 23.12458408 29.67407984 19.05545864 28.04925477
  1.07351323  0.49178857  4.7199511   2.99967732  2.85041903  8.10054442
  0.          6.77465498  6.88599997 11.27768066  2.92685282  0.
  0.          0.          0.         21.19027853 13.5876887  16.56896219
  9.20800095  0.94463993  7.72957811 22.1246005   8.2804662   5.95556162
  8.34158299  1.11868007  0.          0.89817227  6.29339139  0.
  0.         21.4040531   3.76889756  0.          3.28856329  0.
 26.27241658 24.15488007 27.12742429 11.98526099 10.37904431 15.82631576
  6.86629487  7.13781212 29.71849366 20.30900176  0.          0.
  0.          8.31886418  0.          5.76116896  0.          2.60349837
 25.071674

## Kharagpur

In [50]:
predictionK = linReg_output(299, print_=True)  # Kharagpur

[1.46498563e+01 1.46623371e+01 8.28268800e+00 3.95645224e+00
 1.68116850e+00 1.00145964e+01 8.28948664e+00 8.10127785e+00
 8.89069864e+00 4.10650074e+00 2.11272396e+01 7.17916663e+00
 5.38144583e+00 1.63445153e+00 1.33651525e+01 1.89424028e+01
 1.84198478e+01 3.64433987e+01 1.10992675e+01 0.00000000e+00
 1.69060521e+01 1.27283193e+01 0.00000000e+00 4.98608099e+00
 8.67863727e+00 2.34178255e+01 1.19598730e+01 2.00761762e+01
 0.00000000e+00 1.85930539e+01 1.97613378e-02 5.33561903e+00
 9.66421386e+00 1.78521378e+01 0.00000000e+00 0.00000000e+00
 1.34678997e+01 3.21321879e+00 0.00000000e+00 1.18014589e+01
 0.00000000e+00 5.83349711e+00 0.00000000e+00 1.67319201e+01
 3.75413257e+00 1.52517721e+01 1.21585181e+01 1.05071132e+01
 9.38099362e+00 3.64113716e+00 1.65067388e+01 1.77050073e+01
 1.59229063e-01 0.00000000e+00 5.00868386e-01 6.48044138e+00
 0.00000000e+00 1.98381282e+01 0.00000000e+00 4.28413850e+00
 0.00000000e+00 9.78851443e-01 8.01267102e+00 1.71719157e+01
 1.52711464e+01 0.000000

# Predict the rainfall -- 2011

Use the same model to predict the rainfall at these 3 locations on each day of 2011.  Use values in XR as predictors. Compare the results with the true values and compute error for 3 locations separately. [1 marks]

In [71]:
desired_pos = [42, 158, 299]
des = ['Mumbai', 'Delhi', 'Kharagpur']

k=0
for i in desired_pos:
    y_pred = linReg_output(i)
    y_test = table_2011["Pos " + str(i)]
    print(mean_squared_error(y_test, y_pred), ' ||  for position:', des[k])
    k+=1

651.0217026284423  ||  for position: Mumbai
150.43316600138652  ||  for position: Delhi
224.43552088327226  ||  for position: Kharagpur


# Predictor using LASSO

Repeat the same process using LASSO linear regression. Using the coefficients, identify the top 5 predictors for each of the 3 locations. [2 marks]

In [104]:
"""Search for better alpha for lasso"""
# !!! This cell takes long to run

position = "Pos 42"

model = linear_model.Lasso()
y_train = table_2010[position]

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0, 1, 0.01)

# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error',
                      cv=cv, n_jobs=-1)
# perform the search
results = search.fit(table_2010.drop(["Rainy / Not", position], axis = 1),
                     y_train)

# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -10.494
Config: {'alpha': 0.99}


In [128]:
def top5LASSO(position, train=table_2010, test=table_2011, alpha=0.99):

    position = "Pos " + str(position)
    model = linear_model.Lasso(alpha=alpha)
    y_train = train[position]
    model.fit(train.drop(["Rainy / Not", position], axis = 1), y_train)
    y_pred = model.predict(test.drop(["Rainy / Not", position],axis = 1))

    list1 = model.coef_.tolist()

    impFeatures = []
    idxFeatures = []
    for i in range(5):
        imp = list1[max(range(len(list1)), key=list1.__getitem__)]
        idx = max(range(len(list1)), key=list1.__getitem__)
        impFeatures.append(imp)
        idxFeatures.append('Pos' + str(idx))
        list1[idx] = 0

    print("For ", str(position))
    print("-------------------------------")
    print("5 Most important feature are at = ", idxFeatures)
    print("They are: ", impFeatures)    

## MUMBAI

In [129]:
top5LASSO(42)  # MUMBAI

For  Pos 42
-------------------------------
5 Most important feature are at =  ['Pos26', 'Pos25', 'Pos47', 'Pos92', 'Pos90']
They are:  [0.5780270842974066, 0.35411317581165186, 0.3168822235889204, 0.25995559376008776, 0.24819354662788365]


## DELHI

In [131]:
top5LASSO(158)  # DELHI

For  Pos 158
-------------------------------
5 Most important feature are at =  ['Pos131', 'Pos74', 'Pos153', 'Pos202', 'Pos73']
They are:  [0.19749291884217077, 0.15965531926302298, 0.15717432461477387, 0.15668626221270335, 0.12116118722861387]


## KHARAGPUR

In [132]:
top5LASSO(299)  # KHARAGPUR

For  Pos 299
-------------------------------
5 Most important feature are at =  ['Pos50', 'Pos298', 'Pos210', 'Pos291', 'Pos123']
They are:  [0.2187413500717578, 0.18356586231336616, 0.16590108575683313, 0.14663914749747822, 0.10425555408016629]


# Decision Tree

Use Decision Tree on 2010 data to classify each day as 1 or 0 (as given in ZR1). For each day, use the 357-dimensional rainfall vector as feature vector. Report the 10 most discriminative features (i.e. locations) [3 marks]

In [133]:
y_train = table_2010['Rainy / Not']
x_train = table_2010[table_2010.columns[1:]]

clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
feat_importance = clf.tree_.compute_feature_importances(normalize=False)
# print("feat importance = " + str(feat_importance))
important_feature = []
for i in range(len(feat_importance)):
    if feat_importance[i]>0:
        important_feature.append(table_2010.columns[i+1])

In [134]:
important_feature

['Pos 26',
 'Pos 43',
 'Pos 159',
 'Pos 161',
 'Pos 184',
 'Pos 205',
 'Pos 252',
 'Pos 260',
 'Pos 349']

In [135]:
x_train = table_2010[important_feature]
clf = DecisionTreeClassifier()
clf.fit(x_train, y_train)
y_t_pred = clf.predict(x_train)
y_t_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [136]:
print(accuracy_score(y_train,y_t_pred))

1.0


# Classification Using Decision Tree
Use this Decision Tree to classify each day of 2011 as 1 or 0. Report accuracy by comparing with ZR. [1 mark]

In [137]:
x_test = table_2011[important_feature]
y_test = table_2010['Rainy / Not']
y_pred = clf.predict(x_test)
y_pred

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [138]:
print(accuracy_score(y_test,y_pred))

0.6475409836065574
